In [43]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#Reading and parsing the webpage into Python

In [4]:
r = requests.get("https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html")
print(r.text[:200])

<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]> <!--><html lang="en" class="no-js page-interactive section-opinion page-theme-standard tone-opinion page-interactive-default limit-small layout-xlarge app-int


In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

In [6]:
type(soup)

bs4.BeautifulSoup

In [7]:
results = soup.find_all('span', attrs ={'class':'short-desc'})
print(len(results))
print(type(results))
print(results[0])


180
<class 'bs4.element.ResultSet'>
<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>


In [10]:
#Extracting the date
first = results[0]
print(first)


<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>


<strong>Jan. 21 </strong>

In [11]:
first.find('strong')

<strong>Jan. 21 </strong>

In [12]:
first.find('strong').text

'Jan. 21\xa0'

In [15]:
first.find('strong').text[:-1]

'Jan. 21'

In [18]:
first.find('strong').text[:-1]+ ' 2017'

'Jan. 21 2017'

In [19]:
#Extract lie
first.contents

[<strong>Jan. 21 </strong>,
 "“I wasn't a fan of Iraq. I didn't want to go into Iraq.” ",
 <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>]

In [20]:
first.contents[1]

"“I wasn't a fan of Iraq. I didn't want to go into Iraq.” "

In [22]:
first.contents[1][1:-2]

"I wasn't a fan of Iraq. I didn't want to go into Iraq."

In [24]:
#Extracting the statement
first.contents[2]

<span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>

In [25]:
first.contents[2].text

'(He was for an invasion before he was against it.)'

In [30]:
first.contents[2].text[1:-1]

'He was for an invasion before he was against it.'

In [28]:
#Extracting the statement using tag
first.find('a')

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

In [29]:
first.find('a').text

'(He was for an invasion before he was against it.)'

In [31]:
first.find('a').text[1:-1]

'He was for an invasion before he was against it.'

In [32]:
#Extracting the link
first.find('a')

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

In [33]:
first.find('a')['href']

'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'

In [50]:
records = []
for result in results:
    date = result.find('strong').text[:-1]
    lie = result.contents[1][1:-2]
    statement = result.contents[2].text[1:-1]
    link = result.find('a')['href']
    records.append([date, lie, statement, link])
print(f'Total number of records : {len(records)}')
print(f'sample record \n {records[1:3]}')

Total number of records : 180
sample record 
 [['Jan. 21', 'A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.', 'Trump was on the cover 11 times and Nixon appeared 55 times.', 'http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/'], ['Jan. 23', 'Between 3 million and 5 million illegal votes caused me to lose the popular vote.', "There's no evidence of illegal voting.", 'https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html']]


# Storing the data in a csv file


In [52]:
df = pd.DataFrame(records, columns = ['date', 'lie', 'statement', 'link'])
df.head()

,date,lie,statement,link
0,Jan. 21,I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,Jan. 21,A reporter for Time magazine — and I have been...,Trump was on the cover 11 times and Nixon appe...,http://nation.time.com/2013/11/06/10-things-yo...
2,Jan. 23,Between 3 million and 5 million illegal votes ...,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics...
3,Jan. 25,"Now, the audience was the biggest ever. But th...",Official aerial photos show Obama's 2009 inaug...,https://www.nytimes.com/2017/01/21/us/politics...
4,Jan. 25,Take a look at the Pew reports (which show vot...,The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics...


In [53]:
df.to_csv('lies.csv', index=False, encoding='utf-8')

In [54]:
df = pd.read_csv('lies.csv')
df.head()

,date,lie,statement,link
0,Jan. 21,I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,Jan. 21,A reporter for Time magazine — and I have been...,Trump was on the cover 11 times and Nixon appe...,http://nation.time.com/2013/11/06/10-things-yo...
2,Jan. 23,Between 3 million and 5 million illegal votes ...,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics...
3,Jan. 25,"Now, the audience was the biggest ever. But th...",Official aerial photos show Obama's 2009 inaug...,https://www.nytimes.com/2017/01/21/us/politics...
4,Jan. 25,Take a look at the Pew reports (which show vot...,The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics...
